In [2]:
spark = pyspark.sql.SparkSession.builder \
    .master("local") \
    .appName("Titanic") \
    .getOrCreate()

In [3]:
df = spark.read.csv('train.csv', header=True, inferSchema=True)

In [4]:
import pyspark.sql.functions as f  # We tend to need this a lot. Let's import it now.

In [5]:
# Rename label column to make use of PySpark classifier defaults.
df = df.withColumnRenamed('Survived', 'label')
df.show(5)

+-----------+-----+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|label|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+-----+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|    0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|    1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|    1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|    1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|    0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+-----+------+--------------------+------+----+-----+-----+----------------+-------+-----+--

In [6]:
# Let's ensure Age has a non-null value.
df = df.withColumn('Age', f.when(f.col("Age").isNull(), -1).otherwise(f.col("Age")))
df.show(5)

+-----------+-----+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|label|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+-----+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|    0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|    1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|    1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|    1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|    0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+-----+------+--------------------+------+----+-----+-----+----------------+-------+-----+--

In [7]:
# Let's cast the Sex column types to integers so the ML classifier can leverage this data.
df = df.withColumn('Sex',
    f.when(
        f.col('Sex') == 'male', 1
    ).when(
        f.col('Sex') == 'female', 0
    ).otherwise(-1)
)
df.show(5)

+-----------+-----+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|label|Pclass|                Name|Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+-----+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|          1|    0|     3|Braund, Mr. Owen ...|  1|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|    1|     1|Cumings, Mrs. Joh...|  0|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|    1|     3|Heikkinen, Miss. ...|  0|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|    1|     1|Futrelle, Mrs. Ja...|  0|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|    0|     3|Allen, Mr. Willia...|  1|35.0|    0|    0|          373450|   8.05| null|       S|
+-----------+-----+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
only showing top 5 

In [8]:
set(df.select('Embarked').collect())  # Let's see our enum options...

{Row(Embarked='C'), Row(Embarked='Q'), Row(Embarked='S'), Row(Embarked=None)}

In [9]:
# Again, let's cast the Embarked column to something numeric.
df = df.withColumn('Embarked',
    f.when(
        f.col('Embarked') == 'C', 1
    ).when(
        f.col('Embarked') == 'Q', 2
    ).when(
        f.col('Embarked') == 'S', 3
    ).otherwise(-1)  
)
df.show(5)

+-----------+-----+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|label|Pclass|                Name|Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+-----+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|          1|    0|     3|Braund, Mr. Owen ...|  1|22.0|    1|    0|       A/5 21171|   7.25| null|       3|
|          2|    1|     1|Cumings, Mrs. Joh...|  0|38.0|    1|    0|        PC 17599|71.2833|  C85|       1|
|          3|    1|     3|Heikkinen, Miss. ...|  0|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       3|
|          4|    1|     1|Futrelle, Mrs. Ja...|  0|35.0|    1|    0|          113803|   53.1| C123|       3|
|          5|    0|     3|Allen, Mr. Willia...|  1|35.0|    0|    0|          373450|   8.05| null|       3|
+-----------+-----+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
only showing top 5 

In [10]:
df = df.withColumn('Fare', f.when(f.col("Fare").isNull(), -1).otherwise(f.col("Fare")))

In [11]:
# Let's clean up the Cabin column by casting the cabin class to an integer and replacing nulls with -1. 
df = df.withColumn('Cabin', f.lower(f.col('Cabin')))
df = df.withColumn('Cabin', f.substring(f.col('Cabin'), 0, 1))
df = df.withColumn('Cabin', f.ascii(f.col('Cabin')))
df = df.withColumn('Cabin', f.when(f.col("Cabin").isNull(), -1).otherwise(f.col("Cabin")))

df.show(5)

+-----------+-----+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|label|Pclass|                Name|Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+-----+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
|          1|    0|     3|Braund, Mr. Owen ...|  1|22.0|    1|    0|       A/5 21171|   7.25|   -1|       3|
|          2|    1|     1|Cumings, Mrs. Joh...|  0|38.0|    1|    0|        PC 17599|71.2833|   99|       1|
|          3|    1|     3|Heikkinen, Miss. ...|  0|26.0|    0|    0|STON/O2. 3101282|  7.925|   -1|       3|
|          4|    1|     1|Futrelle, Mrs. Ja...|  0|35.0|    1|    0|          113803|   53.1|   99|       3|
|          5|    0|     3|Allen, Mr. Willia...|  1|35.0|    0|    0|          373450|   8.05|   -1|       3|
+-----------+-----+------+--------------------+---+----+-----+-----+----------------+-------+-----+--------+
only showing top 5 

In [12]:
feature_cols = set(df.columns)
feature_cols -= {'PassengerId', 'label', 'Name', 'Ticket'}
feature_cols = list(feature_cols)

In [13]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

features = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",  # PySpark uses this as the default column
).transform(df)

features

DataFrame[PassengerId: int, label: int, Pclass: int, Name: string, Sex: int, Age: double, SibSp: int, Parch: int, Ticket: string, Fare: double, Cabin: int, Embarked: int, features: vector]

In [14]:
train, test = features.randomSplit([1.0, 0.0], seed=2018)
train, test = features.randomSplit([0.7, 0.3], seed=2018)

In [15]:
import pyspark.ml.classification

In [16]:
from pyspark.ml.classification import RandomForestClassifier

classifier = RandomForestClassifier()

model = classifier.fit(train)

predictions = model.transform(test)

In [17]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

print('Test Area Under ROC', BinaryClassificationEvaluator().evaluate(predictions))

Test Area Under ROC 0.8606658047834521


In [19]:
df = spark.read.csv('/Users/paul/Code/Columbia/PDL-Cloudproc/titanic/test.csv', header=True, inferSchema=True)
df = df.withColumnRenamed('Survived', 'label')
df = df.withColumn('Age', f.when(f.col("Age").isNull(), -1).otherwise(f.col("Age")))
df = df.withColumn('Sex',
    f.when(
        f.col('Sex') == 'male', 1
    ).when(
        f.col('Sex') == 'female', 0
    ).otherwise(-1)
)
df = df.withColumn('Embarked',
    f.when(
        f.col('Embarked') == 'C', 1
    ).when(
        f.col('Embarked') == 'Q', 2
    ).when(
        f.col('Embarked') == 'S', 3
    ).otherwise(-1)  
)
df = df.withColumn('Fare', f.when(f.col("Fare").isNull(), -1).otherwise(f.col("Fare")))
df = df.withColumn('Cabin', f.lower(f.col('Cabin')))
df = df.withColumn('Cabin', f.substring(f.col('Cabin'), 0, 1))
df = df.withColumn('Cabin', f.ascii(f.col('Cabin')))
df = df.withColumn('Cabin', f.when(f.col("Cabin").isNull(), -1).otherwise(f.col("Cabin")))

feature_cols = set(df.columns)
feature_cols -= {'PassengerId', 'label', 'Name', 'Ticket'}
feature_cols = list(feature_cols)

features = VectorAssembler(
    inputCols=feature_cols,
    outputCol="features",
).transform(df)

predictions = model.transform(features)

In [20]:
predictions = predictions.withColumnRenamed('prediction', 'Survived')

In [23]:
# predictions.select(['PassengerId', 'Survived']).coalesce(1).write.csv('results.csv')

In [25]:
pddf = predictions.select(['PassengerId', 'Survived']).toPandas()
pddf.head()

,PassengerId,Survived
0,892,0.0
1,893,0.0
2,894,0.0
3,895,0.0
4,896,1.0


In [29]:
pddf['Survived'] = pddf['Survived'].astype('int')
pddf.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [30]:
from datetime import datetime

pddf.to_csv(f'{datetime.now().strftime("%Y-%m-%dT%H:%M")}.csv', index=False, header=True)